In [ ]:
# Install transformers and dataset
# !pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" tensorboard scikit-learn --upgrade --quiet
!pip install matplotlib transformers hf_transfer datasets accelerate evaluate tensorboard scikit-learn kagglehub --upgrade --quiet
# Install git-fls for pushing model and logs to the hugging face hub
# !sudo apt-get install git-lfs --yes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 77.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 80.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from matplotlib import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification

from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import ClassLabel

import os
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"
import kagglehub

In [ ]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("abdoukarimkandji/wolbanking77")
print(dataset_path)

In [3]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77/versions/3/WolBanking77/text/full/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77/versions/3/WolBanking77/text/full/train/train.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [6]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [10]:
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["text"])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
labels = []
for label in tokenized_dataset["train"]:
    labels.append(label['label'])

In [12]:
labels = list(set(labels))
len(labels)

77

In [14]:
from datasets import ClassLabel
new_features = tokenized_dataset['train'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['train'] = tokenized_dataset['train'].cast(new_features)

Casting the dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [16]:
new_features = tokenized_dataset['test'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['test'] = tokenized_dataset['test'].cast(new_features)

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForSequenceClassification
# Prepare model labels - useful for inference
# labels = tokenized_dataset["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/mDeBERTa-v3-base-mnli-xnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([77]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([77, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [29]:
# Free up memory before re-loading the model.
# del model, trainer
# del trainer
# torch.cuda.empty_cache()

In [19]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Id for remote repository
repository_id = "mDeBERTa-v3-base-banking77-wolof"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8, # 8 16
    per_device_eval_batch_size=8, # 8
    learning_rate=2e-05,
	num_train_epochs=20, # 5
    warmup_ratio=0.1,
    weight_decay=0.06,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
	# torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    remove_unused_columns=False

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [20]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,4.335100,4.266982,0.010054
2,3.961100,3.752935,0.028131
3,3.571500,3.267120,0.111471
4,2.946200,2.718289,0.234354
5,2.525100,2.342991,0.312050
6,2.065900,2.063534,0.375551
7,1.791500,1.791389,0.451480
8,1.438700,1.627523,0.515804
9,1.231400,1.490265,0.585771
10,1.036200,1.421505,0.593401


TrainOutput(global_step=10000, training_loss=1.4848422714233398, metrics={'train_runtime': 11117.2096, 'train_samples_per_second': 7.196, 'train_steps_per_second': 0.9, 'total_flos': 2.106343612416e+16, 'train_loss': 1.4848422714233398, 'epoch': 20.0})

In [21]:
# 'eval_f1': 0.34
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 1.3746931552886963,
 'eval_f1': 0.6906444951204278,
 'eval_runtime': 36.2203,
 'eval_samples_per_second': 27.609,
 'eval_steps_per_second': 3.451,
 'epoch': 20.0}

In [22]:
# Save processor and create model card
tokenizer.save_pretrained(repository_id)

('mDeBERTa-v3-base-banking77-wolof/tokenizer_config.json',
 'mDeBERTa-v3-base-banking77-wolof/special_tokens_map.json',
 'mDeBERTa-v3-base-banking77-wolof/spm.model',
 'mDeBERTa-v3-base-banking77-wolof/added_tokens.json',
 'mDeBERTa-v3-base-banking77-wolof/tokenizer.json')

In [23]:
model_id = "/workspace/NLP_TASKS/mDeBERTa-v3-base-banking77-wolof/checkpoint-10000"
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/mDeBERTa-v3-base-banking77-wolof")
tokenizer = AutoTokenizer.from_pretrained(repository_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True).to("cuda")

In [24]:
device = "cuda"
# import torch
def predict(batch, model):
    input_ids = torch.tensor(batch['input_ids']).to(device).unsqueeze(0)
    token_type_ids = torch.tensor(batch['token_type_ids']).to(device).unsqueeze(0)
    attention_mask = torch.tensor(batch['attention_mask']).to(device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).logits
        predicted = torch.argmax(logits, dim=-1)[0]
    return predicted

In [25]:
model.to("cuda")
predictions = [predict(batch, model) for batch in tokenized_dataset["test"]]

In [26]:
predictions[0]

tensor(19, device='cuda:0')

In [27]:
y_pred = []
for y in predictions:
    y_pred.append(labels[y])
y_pred

['topping up by card',
 'wrong amount of cash received',
 'transfer fee charged',
 'disposable card limits',
 'card delivery estimate',
 'card delivery estimate',
 'exchange charge',
 'declined transfer',
 'card not working',
 'verify source of funds',
 'getting spare card',
 'extra charge on statement',
 'pending card payment',
 'top up by cash or cheque',
 'declined transfer',
 'cash withdrawal not recognised',
 'verify my identity',
 'top up reverted',
 'card payment wrong exchange rate',
 'terminate account',
 'card payment not recognised',
 'wrong amount of cash received',
 'declined transfer',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'why verify identity',
 'transfer timing',
 'cancel transfer',
 'request refund',
 'verify my identity',
 'why verify identity',
 'top up failed',
 'virtual card not working',
 'pending cash withdrawal',
 'pending cash withdrawal',
 'fiat currency support',
 'country support',
 'card about to expire',
 'transfer not received b

In [28]:
y_test = []
for y in tokenized_dataset["test"]['label']:
    y_test.append(labels[y])
y_test

['topping up by card',
 'Refund not showing up',
 'transfer fee charged',
 'get disposable virtual card',
 'card arrival',
 'card delivery estimate',
 'exchange charge',
 'card delivery estimate',
 'transaction charged twice',
 'verify source of funds',
 'order physical card',
 'extra charge on statement',
 'declined card payment',
 'apple pay or google pay',
 'top up reverted',
 'cash withdrawal not recognised',
 'verify my identity',
 'wrong amount of cash received',
 'card payment wrong exchange rate',
 'terminate account',
 'card payment not recognised',
 'wrong amount of cash received',
 'balance not updated after cheque or cash deposit',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'why verify identity',
 'transfer timing',
 'cancel transfer',
 'request refund',
 'verify my identity',
 'verify my identity',
 'top up reverted',
 'virtual card not working',
 'wrong amount of cash received',
 'pending cash withdrawal',
 'fiat currency support',
 'country support'

In [29]:
from sklearn import metrics
# y_test = test_df['label']
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.86      0.71      0.77        17
                                activate my card       0.43      0.62      0.51        16
                                       age limit       1.00      0.80      0.89        10
                         apple pay or google pay       1.00      0.79      0.88        14
                                     atm support       0.80      0.89      0.84         9
                                automatic top up       0.92      0.86      0.89        14
         balance not updated after bank transfer       0.43      0.53      0.47        17
balance not updated after cheque or cash deposit       0.80      0.71      0.75        17
                         beneficiary not allowed       0.33      0.27      0.30        15
                                 cancel transfer       0.68      0.93      0.79        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.6904386296505297

In [31]:
metrics.accuracy_score(y_test, y_pred)

0.692

In [32]:
metrics.precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.7095694021378856

In [33]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.692

In [34]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [35]:
compute_metrics(y_test, y_pred)

f1_score: 0.6904386296505297
precision_score: 0.7095694021378856
recall_score: 0.692


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
